# **Library**

In [ ]:
import pandas as pd
import requests
import yfinance as yf
import numpy as np
from datetime import datetime

# **Teknikal**

In [ ]:
ticker = "BBCA.JK"
data = yf.download(ticker, start="2021-01-01", end=datetime.now().strftime('%Y-%m-%d'), progress=False)

print(f"Data scraped: {len(data)} rows from 2021 to {data.index[-1].date()}")

/tmp/ipython-input-1557031061.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start="2021-01-01", end=datetime.now().strftime('%Y-%m-%d'), progress=False)


Data scraped: 1232 rows from 2021 to 2026-02-10


In [ ]:
info = yf.Ticker(ticker).info

In [ ]:
def compute_rsi(series, period=14):
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

def bollinger_bands(series, period=20, std_dev=2):
    sma = series.rolling(period).mean()
    std = series.rolling(period).std()
    upper = sma + (std * std_dev)
    lower = sma - (std * std_dev)
    return upper, lower

def macd(series, fast=12, slow=26, signal=9):
    ema_fast = series.ewm(span=fast).mean()
    ema_slow = series.ewm(span=slow).mean()
    macd_line = ema_fast - ema_slow
    signal_line = macd_line.ewm(span=signal).mean()
    return macd_line, signal_line

In [ ]:
data['SMA_5'] = data['Close'].rolling(5).mean()
data['SMA_20'] = data['Close'].rolling(20).mean()
data['RSI_14'] = compute_rsi(data['Close'], 14)
data['BB_upper'], data['BB_lower'] = bollinger_bands(data['Close'], 20, 2)
data['MACD'], data['MACD_signal'] = macd(data['Close'])

In [ ]:
data.dropna(inplace=True)

In [ ]:
data_csv = data.reset_index()

In [ ]:
technical_csv = 'BBCA_technical_cleaned.csv'
data_csv.to_csv(technical_csv, index=False)
print(f"Technical data saved: {technical_csv} ({len(data_csv)} rows)")

Technical data saved: BBCA_technical_cleaned.csv (1213 rows)


# **Fundamental**

In [ ]:
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                  "(KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36"
}

def fetch_first_table(url: str) -> pd.DataFrame:
    r = requests.get(url, headers=HEADERS, timeout=30)
    r.raise_for_status()
    tables = pd.read_html(r.text)
    if not tables:
        raise ValueError(f"No tables found at {url}")
    return tables[0]

def _index_to_datetime(idx) -> pd.DatetimeIndex:
    if isinstance(idx, pd.MultiIndex):
        base = idx.get_level_values(-1).astype(str)
    else:
        base = idx.astype(str)

    extracted = pd.Series(base, index=idx).str.extract(r"([A-Za-z]{3}\s+\d{1,2},\s+\d{4})", expand=False)

    cleaned = extracted.fillna(pd.Series(base, index=extracted.index))

    return pd.to_datetime(cleaned, errors="coerce")

def table_to_timeseries(df_raw: pd.DataFrame) -> pd.DataFrame:
    df = df_raw.copy()

    first_col = df.columns[0]
    df = df.set_index(first_col)

    df = df.T

    dt_idx = _index_to_datetime(df.index)
    keep = ~dt_idx.isna()
    df = df.loc[keep].copy()
    df.index = dt_idx[keep]
    df.index.name = "date"

    df.columns = (
        df.columns.astype(str)
        .str.strip()
        .str.lower()
        .str.replace(r"[^a-z0-9]+", "_", regex=True)
        .str.replace(r"_$", "", regex=True)
    )

    def to_number(x):
        if pd.isna(x):
            return pd.NA
        s = str(x).strip()
        if s in {"-", "—", ""}:
            return pd.NA
        neg = s.startswith("(") and s.endswith(")")
        s = s.replace(",", "").replace("(", "").replace(")", "")
        mult = 1.0
        if s.endswith("B"):
            mult = 1e9
            s = s[:-1]
        elif s.endswith("M"):
            mult = 1e6
            s = s[:-1]
        try:
            val = float(s) * mult
            return -val if neg else val
        except Exception:
            return pd.NA

    for c in df.columns:
        df[c] = df[c].map(to_number)

    return df

def pick_col(df: pd.DataFrame, candidates: list[str]) -> pd.Series:
    for c in candidates:
        if c in df.columns:
            return df[c]
    for c in df.columns:
        for k in candidates:
            if k in c:
                return df[c]
    return pd.Series([pd.NA] * len(df), index=df.index)

def compute_growth(series: pd.Series, periods: int) -> pd.Series:
    return series.astype("float64").pct_change(periods=periods)

In [ ]:
url_bs = "https://stockanalysis.com/quote/idx/BBCA/financials/balance-sheet/?p=quarterly"
url_is = "https://stockanalysis.com/quote/idx/BBCA/financials/?p=quarterly"

bs_raw = fetch_first_table(url_bs)
is_raw = fetch_first_table(url_is)

bs = table_to_timeseries(bs_raw)
inc = table_to_timeseries(is_raw)

/tmp/ipython-input-1423726715.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(r.text)
/tmp/ipython-input-1423726715.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(r.text)


In [ ]:
df_fund = pd.DataFrame(index=bs.index.union(inc.index)).sort_index()
df_fund.index.name = "date"

In [ ]:
# Balance sheet
df_fund["total_assets"] = pick_col(bs, ["total_assets"])
df_fund["total_liabilities"] = pick_col(bs, ["total_liabilities"])
df_fund["total_equity"] = pick_col(bs, ["total_equity", "total_stockholders_equity", "total_shareholders_equity"])

df_fund["cash_and_cash_equivalents"] = pick_col(bs, ["cash_and_cash_equivalents", "cash_cash_equivalents"])
df_fund["short_term_debt"] = pick_col(bs, ["short_term_debt", "current_debt", "short_long_term_debt"])
df_fund["long_term_debt"] = pick_col(bs, ["long_term_debt", "long_term_debt_and_capital_lease_obligation"])

In [ ]:
# Income statement
df_fund["revenue"] = pick_col(inc, ["revenue", "total_revenue"])
df_fund["net_income"] = pick_col(inc, ["net_income", "net_income_common_stockholders", "net_income_to_common"])

In [ ]:
# --- Rasio ---
df_fund["roe"] = df_fund["net_income"] / df_fund["total_equity"]
df_fund["roa"] = df_fund["net_income"] / df_fund["total_assets"]
df_fund["debt_to_equity"] = df_fund["total_liabilities"] / df_fund["total_equity"]

In [ ]:
# --- Growth QoQ & YoY ---
df_fund["revenue_qoq"] = compute_growth(df_fund["revenue"], periods=1)
df_fund["net_income_qoq"] = compute_growth(df_fund["net_income"], periods=1)
df_fund["revenue_yoy"] = compute_growth(df_fund["revenue"], periods=4)
df_fund["net_income_yoy"] = compute_growth(df_fund["net_income"], periods=4)

In [ ]:
df_fund = df_fund[df_fund.index >= pd.Timestamp("2021-01-01")]

In [ ]:
cols = [
    "total_assets","total_liabilities","total_equity",
    "cash_and_cash_equivalents","short_term_debt","long_term_debt",
    "revenue","net_income",
    "roe","roa","debt_to_equity",
    "revenue_qoq","net_income_qoq","revenue_yoy","net_income_yoy",
]
df_fund = df_fund[cols].sort_index()

In [ ]:
df_fund = df_fund.reset_index()

In [ ]:
cols_to_drop = [col for col in df_fund.columns if df_fund[col].isna().all()]
df_fund = df_fund.drop(columns=cols_to_drop)

In [ ]:
print("Dropped columns:", cols_to_drop)

Dropped columns: ['total_equity', 'cash_and_cash_equivalents', 'short_term_debt', 'roe', 'debt_to_equity']


In [ ]:
df_fund.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   date               20 non-null     datetime64[ns]
 1   total_assets       20 non-null     float64       
 2   total_liabilities  20 non-null     float64       
 3   long_term_debt     20 non-null     float64       
 4   revenue            20 non-null     float64       
 5   net_income         20 non-null     float64       
 6   roa                20 non-null     float64       
 7   revenue_qoq        19 non-null     float64       
 8   net_income_qoq     19 non-null     float64       
 9   revenue_yoy        16 non-null     float64       
 10  net_income_yoy     16 non-null     float64       
dtypes: datetime64[ns](1), float64(10)
memory usage: 1.8 KB


In [ ]:
output_path = "bbca_fund_quarterly.csv"
df_fund.to_csv(output_path, index=False)

print(f"CSV quarterly berhasil dibuat: {output_path}")

CSV quarterly berhasil dibuat: bbca_fund_quarterly.csv


# **Merge**

In [ ]:
fund = pd.read_csv('bbca_fund_quarterly.csv')
tech = pd.read_csv('BBCA_technical_cleaned.csv')

In [ ]:
# parse tanggal
fund["date"] = pd.to_datetime(fund["date"])
tech["Date"] = pd.to_datetime(tech["Date"])

In [ ]:
# buat kolom quarter (Q1, Q2, dst)
fund["quarter"] = fund["date"].dt.to_period("Q")
tech["quarter"] = tech["Date"].dt.to_period("Q")

In [ ]:
# merge: setiap baris teknikal akan dapat fundamental dari kuartal yang sama
merged = tech.merge(
    fund.drop(columns=["date"]),   # date fund tidak perlu (sudah diwakili quarter)
    on="quarter",
    how="left"
)

# rapihin: quarter jadi string biar gampang disimpan/di-read ulang
merged["quarter"] = merged["quarter"].astype(str)

In [ ]:
core_cols = ["total_assets","total_liabilities","revenue","net_income","roa", "long_term_debt"]
qoq_cols  = ["revenue_qoq","net_income_qoq"]
yoy_cols  = ["revenue_yoy","net_income_yoy"]

growth_cols = qoq_cols + yoy_cols

fund_cols = core_cols + growth_cols

In [ ]:
merged.columns

Index(['Date', 'Close', 'High', 'Low', 'Open', 'Volume', 'SMA_5', 'SMA_20',
       'RSI_14', 'BB_upper', 'BB_lower', 'MACD', 'MACD_signal', 'quarter',
       'total_assets', 'total_liabilities', 'long_term_debt', 'revenue',
       'net_income', 'roa', 'revenue_qoq', 'net_income_qoq', 'revenue_yoy',
       'net_income_yoy'],
      dtype='object')

## Dataset A

Data fundamental tahun 2021 yang tidak memiliki nilai (NaN) di isi dengan nilai 0 dimana data yang kosong adalah data qoq dan yoy. Data fundamental tahun 2026 tidak memiliki nilai karena belum rilis sehingga digunakan forward fill.

In [ ]:
dfA = merged.copy()

# fund_available sebelum ffill
dfA["fund_available"] = dfA[core_cols].notna().all(axis=1).astype(int)
dfA["qoq_available"]  = dfA[qoq_cols].notna().all(axis=1).astype(int)
dfA["yoy_available"]  = dfA[yoy_cols].notna().all(axis=1).astype(int)

In [ ]:
# ffill ke 2026
dfA = dfA.sort_values("Date")
dfA[fund_cols] = dfA[fund_cols].ffill()

In [ ]:
# isi growth NaN (yang tersisa) jadi 0
dfA[growth_cols] = dfA[growth_cols].fillna(0)
dfA = dfA.drop(columns=['quarter'])

In [ ]:
dfA.to_csv("merged_Dataset_A.csv", index=False)

## Dataset B

Data fundamental tahun 2021 yang NaN dibiarkan NaN untuk jadi opsi kedua jika pada tahap modeling ingin menggunakan model yang bisa mengatasi data NaN. Data fundamental tahun 2026 tidak memiliki nilai karena belum rilis sehingga digunakan forward fill.

In [ ]:
dfB = merged.copy()

# fund_available dibuat SEBELUM ffill
dfB["fund_available"] = dfB[core_cols].notna().all(axis=1).astype(int)
dfB["qoq_available"]  = dfB[qoq_cols].notna().all(axis=1).astype(int)
dfB["yoy_available"]  = dfB[yoy_cols].notna().all(axis=1).astype(int)

In [ ]:
# ffill ke 2026
dfB = dfB.sort_values("Date")
dfB[fund_cols] = dfB[fund_cols].ffill()
dfB = dfB.drop(columns=['quarter'])

In [ ]:
dfB.to_csv("merged_Dataset_B.csv", index=False)